In [1]:
import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.layers import Embedding, LSTM, Dense, Dropout, SpatialDropout1D, Bidirectional
import tensorflow as tf
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression
import re
import numpy as np

In [2]:
from scikeras.wrappers import KerasClassifier
# model = keras.models.load_model('m.h5')
# customModel = KerasClassifier(model = model, epochs=100, batch_size=16, verbose=1)


In [16]:
def commentCleaner(comments):
    cleaned_comments = []
    for comment in comments:
        # Remove special symbols, emojis, reddit username mentions, and hyperlinks
        comment = re.sub(r"[^\w\s]|http\S+|www\S+|u/[A-Za-z0-9_-]+", "", comment)
        comment = comment.lower()
        # Tokenize the comment
        tokens = comment.split()
        # tokens = comment.split(' ')
        # Remove stop words
        stop_words = set(stopwords.words("english"))
        tokens = [token for token in tokens if token not in stop_words]
        
        lemmatizer = WordNetLemmatizer()
        tokens = [lemmatizer.lemmatize(token) for token in tokens]
        
        # Join the tokens back into a single string
        cleaned_comment = " ".join(tokens)
        cleaned_comments.append(cleaned_comment)
        
    
    return cleaned_comments

In [18]:
data = pd.read_csv("augmented_data.csv")
cleaned_data = data.copy()
cleaned_data["Sentence"] = commentCleaner(cleaned_data["Sentence"])
cleaned_data.head()

,Unnamed: 0,Sentence,Sentiment
0,0,geosolutions technology leverage benefon gps s...,positive
1,1,esi low 150 250 bk real possibility,negative
2,2,last quarter 2010 componenta net sale doubled ...,positive
3,3,according finnishrussian chamber commerce majo...,neutral
4,4,swedish buyout firm sold remaining 224 percent...,neutral


In [21]:
# Converting the integer labels to onehot encoding]
data = pd.read_csv('augmented_data.csv')
y = pd.get_dummies(data['Sentiment'])
print(y.head())
X = data["Sentence"]
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.3, random_state = 42)
print(data.head())
print(y_train.head())
tokenizerFinal = Tokenizer(num_words=1000, split=' ') 
print(cleaned_data['Sentence'].values)
tokenizerFinal.fit_on_texts(cleaned_data['Sentence'].values)
X_train

   negative  neutral  positive
0     False    False      True
1      True    False     False
2     False    False      True
3     False     True     False
4     False     True     False
   Unnamed: 0                                           Sentence Sentiment
0           0  The GeoSolutions technology will leverage Bene...  positive
1           1  $ESI on lows, down $1.50 to $2.50 BK a real po...  negative
2           2  For the last quarter of 2010 , Componenta 's n...  positive
3           3  According to the Finnish-Russian Chamber of Co...   neutral
4           4  The Swedish buyout firm has sold its remaining...   neutral
      negative  neutral  positive
3208     False    False      True
4343     False     True     False
2153     False     True     False
5551      True    False     False
6571      True    False     False
['geosolutions technology leverage benefon gps solution providing location based search technology community platform location relevant multimedia content new p

3208    - I am pleased that Bjorn Wahlroos has accepte...
4343    Auburn 's sales in 2007 were CAD 41 million ( ...
2153    Profit for the period totalled EUR 1.1 mn , do...
5551    $ACOM http://stks.co/1G6x Downside breakout lo...
6571    78 users on Vetr are bearish on Tesla Motors, ...
                              ...                        
3772    @BULLYA @pollux654321 My 50 $KORS 80 Calls are...
5191    According to Sepp+ñnen , the new technology UM...
5226     $CRUS Upgraded to a buy by Alpha Street Research
5390    Favourable currency rates also contributed to ...
860     Tesco breaks its downward slide by cutting sal...
Name: Sentence, Length: 4635, dtype: object

In [23]:
customModel.initialize(X_train, y_train)

ValueError: Expected 2D array, got 1D array instead:
array=['- I am pleased that Bjorn Wahlroos has accepted the nomination .'
 "Auburn 's sales in 2007 were CAD 41 million ( approximately EUR 27 million ) , and the company employs some 150 people ."
 'Profit for the period totalled EUR 1.1 mn , down from EUR 1.6 mn in the third quarter of 2008 .'
 ... '$CRUS Upgraded to a buy by Alpha Street Research'
 "Favourable currency rates also contributed to higher net sales , '' CEO Kari Kauniskangas said ."
 'Tesco breaks its downward slide by cutting sales decline in half'].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

In [28]:
from joblib import load
from keras.models import load_model
pipeline = load('/Users/aadeesh/redditSentiment/pipeline.pkl')
pipeline.named_steps['model'].model = load_model('m.h5')


ModuleNotFoundError: No module named '__builtin__'

In [62]:
from scikeras.wrappers import KerasClassifier

def classification_model():
    # Building our model
    model = keras.Sequential()
    model.add(Embedding(1000, 120, input_length = 31))
    model.add(SpatialDropout1D(0.4))
    model.add(Bidirectional(LSTM(64, dropout=0.2)))
    model.add(Dense(32, activation='relu'))
    model.add(Dropout(0.1))
    model.add(Dense(16, activation='relu'))
    model.add(Dropout(0.1))
    model.add(Dense(8, activation='relu'))
    model.add(Dropout(0.1))
    model.add(Dense(3,activation='softmax'))
    model.compile(loss = 'categorical_crossentropy', optimizer='adam', metrics = ['accuracy'])
    return model

model = classification_model


In [53]:
# Cleans the sentence of links and emojis, removes stop words, and lemmatizes words
def commentCleaner(comments):
    cleaned_comments = []
    for comment in comments:
        # Remove special symbols, emojis, reddit username mentions, and hyperlinks
        comment = re.sub(r"[^\w\s]|http\S+|www\S+|u/[A-Za-z0-9_-]+", "", comment)
        comment = comment.lower()
        # Tokenize the comment
        tokens = comment.split()
        # tokens = comment.split(' ')
        # Remove stop words
        stop_words = set(stopwords.words("english"))
        tokens = [token for token in tokens if token not in stop_words]
        
        lemmatizer = WordNetLemmatizer()
        tokens = [lemmatizer.lemmatize(token) for token in tokens]
        
        # Join the tokens back into a single string
        cleaned_comment = " ".join(tokens)
        cleaned_comments.append(cleaned_comment)
        
    
    return cleaned_comments

print(commentCleaner(["One of the other reviewers mentioned watching 1 oz episode"]))


['one reviewer mentioned watching 1 oz episode']


In [56]:
def tokenizeComments(comments, tokenizer):
    print("Comments recieved for tokenization: ")
    print(comments)
    # tokenizer = Tokenizer(num_words=1000, split=' ') 
    # tokenizer.fit_on_texts(comments)
    print("Fitted tokenizer to sample texts")
    tokenized_comments = tokenizer.texts_to_sequences(comments)
    print("Converted to sequences")
    tokenized_comments = pad_sequences(tokenized_comments, 31)
    print("Padded succesfully")
    print(tokenized_comments)
    return tokenized_comments


In [94]:
import dill as pickle

def load_pipeline_keras(cleaner, model, tokenizer, folder_name="model"):
    cleaner = pickle.load(open(folder_name+'/'+cleaner,'rb'))
    tokenizerFinal = pickle.load(open(folder_name+'/'+tokenizer,'rb'))
    build_model = keras.models.load_model(folder_name+'/'+model)
    cleaner.tokenizer = tokenizerFinal
    # classifier = KerasClassifier(model=build_model, epochs=1, batch_size=10, verbose=1)
    # classifier.classes_ = pickle.load(open(folder_name+'/'+classes,'rb'))
    # classifier.model = build_model
    # build_model.compile(loss = 'categorical_crossentropy', optimizer='adam', metrics = ['accuracy'])
    return Pipeline([
        ('cleaner', cleaner),
        ('lstm', build_model)
    ])



In [95]:
pipeline = load_pipeline_keras('transformer.pkl', 'lstm.h5', 'tokenizer.pkl')
pipeline

Pipeline(steps=[('cleaner',
                 textTransformer(tokenizer=<keras.src.preprocessing.text.Tokenizer object at 0x151fcab20>)),
                ('lstm',
                 <keras.src.engine.sequential.Sequential object at 0x151faf580>)])

In [96]:

pred = pipeline.predict(['Well Apple actually makes quality products as opposed to those others'])

print(pred)


Starting transform
['Well Apple actually makes quality products as opposed to those others']
Cleaned comments
Starting tokenization
Comments recieved for tokenization: 
['well apple actually make quality product opposed others']
Fitted tokenizer to sample texts
Converted to sequences
Padded succesfully
[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0  76 659 186 745  37]]
Tokenized
Ending transform
1/1 [==============================] - 1s 508ms/step
[[0.16205938 0.6267933  0.21114734]]


In [51]:
pipeline.steps

[('cleaner',
  textTransformer(tokenizer=<keras.src.preprocessing.text.Tokenizer object at 0x299992be0>)),
 ('lstm',
  KerasClassifier(
  	model=<keras.src.engine.sequential.Sequential object at 0x299a01940>
  	build_fn=<function load_pipeline_keras.<locals>.<lambda> at 0x2999971f0>
  	warm_start=False
  	random_state=None
  	optimizer=rmsprop
  	loss=None
  	metrics=None
  	batch_size=10
  	validation_batch_size=None
  	verbose=1
  	callbacks=None
  	validation_split=0.0
  	shuffle=True
  	run_eagerly=False
  	epochs=1
  	class_weight=None
  ))]